<a href="https://colab.research.google.com/github/HosungJung01/AI-python-connect/blob/master/Lab14F0_%EB%B3%B5%EC%A7%80%EB%8D%B0%EC%9D%B4%ED%84%B0_oneHotEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
import tensorflow as tf
tf.random.set_seed(777)  # for reproducibility
print(tf.__version__)

2.2.0


In [0]:
np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)

### 1.2019년 가계복지데이터를 이용하여 성별, 나이, 소득, 종교여부, 흡연상태, 결혼상태에 관한 데이터를 추출합니다.(10점)
이용자료 : koweps_hpwc14_2019_beta1.sas7bdat

In [0]:
df = pd.read_csv("C:/HSCHUNG/PYTHON/B_CLASS/A_BigDataEconomics/DATA/C_KOWEPS/A_koweps01.csv")

FileNotFoundError: ignored

In [0]:
df['age'] = (2020 - df['birth']).astype('int')

In [0]:
# marriage = 혼인상태 0.비해당(18세 미만) 1.유배우 2.사별 3.이혼 4.별거 5.미혼(18세이상, 미혼모 포함) 6.기타(사망 등)
# 1. 유배우, 3, 이혼, 4, 별거, 5. 미혼 만 남김

In [0]:
df1 = df[df['marriage'].isin([1,3,4,5])]

In [0]:
df1

,sex,birth,number,education,marriage,religion,region,job,income,somking,education_year,age
0,1.0,1942,2.0,7.0,3.0,1.0,1.0,762.0,108.9,2.0,16,78
2,1.0,1978,4.0,7.0,1.0,2.0,1.0,312.0,322.0,1.0,16,42
3,2.0,1975,4.0,7.0,1.0,2.0,1.0,254.0,120.0,2.0,16,45
4,1.0,1975,1.0,7.0,5.0,1.0,1.0,286.0,300.0,2.0,16,45
5,2.0,1961,3.0,4.0,1.0,2.0,1.0,951.0,100.0,2.0,9,59
...,...,...,...,...,...,...,...,...,...,...,...,...
4502,1.0,1966,6.0,5.0,1.0,1.0,6.0,874.0,230.0,2.0,12,54
4503,2.0,1962,6.0,7.0,1.0,1.0,6.0,899.0,138.0,2.0,16,58
4504,2.0,1993,1.0,5.0,5.0,2.0,5.0,411.0,286.0,2.0,12,27
4505,1.0,1956,2.0,5.0,1.0,1.0,6.0,854.0,179.0,2.0,12,64


In [0]:
# 1. marriaged, 2. single
def status(x):
    if x==1:
        return 1
    elif x==3:
        return 2
    elif x==4:
        return 3
    else:
        return 4

In [0]:
df['marriage_type'] = df['marriage'].apply(status)

In [0]:
df['age2'] = df['age']**2
df['job_code'] = (df['job']/100).astype(int)
df['sex'] = (df['sex']).astype(int)
df['smoking'] = (df['somking']).astype(int)
df['religion'] = (df['religion']).astype(int)

In [0]:
sex_dummies = pd.get_dummies(df['sex']).rename(columns=lambda x: 's'+str(x))
smoking_dummies = pd.get_dummies(df['smoking']).rename(columns=lambda x: 'k'+str(x))
job_dummies = pd.get_dummies(df['job_code']).rename(columns=lambda x: 'd'+str(x))
religion_dummies = pd.get_dummies(df['religion']).rename(columns=lambda x: 'r'+str(x))

In [0]:
df1 = pd.concat([df, religion_dummies, sex_dummies, smoking_dummies], axis=1)

In [0]:
df1

,sex,birth,number,education,marriage,religion,region,job,income,somking,...,marriage_type,age2,job_code,smoking,r1,r2,s1,s2,k1,k2
0,1,1942,2.0,7.0,3.0,1,1.0,762.0,108.9,2.0,...,2,6084,7,2,1,0,1,0,0,1
1,2,1940,1.0,3.0,2.0,1,2.0,999.0,20.0,2.0,...,4,6400,9,2,1,0,0,1,0,1
2,1,1978,4.0,7.0,1.0,2,1.0,312.0,322.0,1.0,...,1,1764,3,1,0,1,1,0,1,0
3,2,1975,4.0,7.0,1.0,2,1.0,254.0,120.0,2.0,...,1,2025,2,2,0,1,0,1,0,1
4,1,1975,1.0,7.0,5.0,1,1.0,286.0,300.0,2.0,...,4,2025,2,2,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4502,1,1966,6.0,5.0,1.0,1,6.0,874.0,230.0,2.0,...,1,2916,8,2,1,0,1,0,0,1
4503,2,1962,6.0,7.0,1.0,1,6.0,899.0,138.0,2.0,...,1,3364,8,2,1,0,0,1,0,1
4504,2,1993,1.0,5.0,5.0,2,5.0,411.0,286.0,2.0,...,4,729,4,2,0,1,0,1,0,1
4505,1,1956,2.0,5.0,1.0,1,6.0,854.0,179.0,2.0,...,1,4096,8,2,1,0,1,0,0,1


### 2. 소득, 나이, 나이2, 교육연수, 셩별더미, 흡연여부더미, 종교여부 더미를 X  변수로, 결혼여부를 Y 변수로 두고 Logistic Regression을 실시하고 결과를 해석합니다.(20점)

# 새 섹션

In [0]:
x_data =df1[['income', 'age', 'age2', 'education_year', 's1', 'k1', 'r1']].to_numpy()

In [0]:
#y_data =df1['marriage_type'].to_numpy()
y_data =df1[['marriage_type']].to_numpy()

### 3. Tensorflow를 통한 계수 및 정확도 추정

<h4> 가. 데이타

In [0]:
#convert into numpy and float format
X = np.asarray(x_data, dtype=np.float32)
y = np.asarray(y_data, dtype=np.float32)

In [0]:
print(X.shape)
print(y.shape)

(4507, 7)
(4507, 1)


In [0]:
k = x_data.shape[1]

In [0]:
k

7

In [0]:
nb_classes = 4  # 1-4

In [0]:
Y_one_hot = tf.one_hot(y.astype(np.int32), nb_classes)

In [0]:
Y_one_hot

<tf.Tensor: shape=(4507, 1, 4), dtype=float32, numpy=
array([[[0., 0., 1., 0.]],

       [[0., 0., 0., 0.]],

       [[0., 1., 0., 0.]],

       ...,

       [[0., 0., 0., 0.]],

       [[0., 1., 0., 0.]],

       [[0., 0., 0., 0.]]], dtype=float32)>

In [0]:
Y_one_hot = tf.reshape(Y_one_hot,[4507, -1] )

In [0]:
Y_one_hot

<tf.Tensor: shape=(4507, 4), dtype=float32, numpy=
array([[0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

In [0]:
X.shape

(4507, 7)

In [0]:
k

7

In [0]:
W = tf.Variable(tf.random.normal((k, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')

learning_rate = tf.Variable(0.001)

for i in range(500000+1):
    with tf.GradientTape() as tape:
        
        logits = tf.matmul(X, W) + b
        
        hypothesis  = tf.nn.softmax(logits)
        
        cost_i = tf.keras.losses.categorical_crossentropy(y_true=Y_one_hot, y_pred=logits, from_logits=True) 
        cost = tf.reduce_mean(cost_i) 

        W_grad, b_grad = tape.gradient(cost, [W, b])
        
        W.assign_sub(learning_rate * W_grad)
        b.assign_sub(learning_rate * b_grad)
        
        predicted = tf.argmax(hypothesis, 1)
        correct_prediction = tf.equal(predicted, tf.argmax(Y_one_hot,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))
    
    if i % 50000 == 0:
        print("Steps: {:5} | Costs: {:10.6f} | Accuracy {:10.6f}".format(i, cost.numpy(), accuracy.numpy()))

Steps:     0 | Costs: 2320.653564 | Accuracy   0.288440
Steps: 50000 | Costs: 649.624939 | Accuracy   0.104060
Steps: 100000 | Costs: 223.137314 | Accuracy   0.639006
Steps: 150000 | Costs: 568.246521 | Accuracy   0.345463
Steps: 200000 | Costs: 260.699371 | Accuracy   0.432660
Steps: 250000 | Costs: 103.910416 | Accuracy   0.622365
Steps: 300000 | Costs: 277.779449 | Accuracy   0.644997
Steps: 350000 | Costs: 294.088318 | Accuracy   0.648990
Steps: 400000 | Costs: 526.879944 | Accuracy   0.644997
Steps: 450000 | Costs: 433.266754 | Accuracy   0.123807
Steps: 500000 | Costs: 248.028397 | Accuracy   0.644997
